In [1]:

import numpy as np
import pandas as pd


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover

In [4]:
appName="Sentimen Analysis Spark"
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [5]:
tweet_csv=spark.read.csv("tweets.csv", inferSchema=True, header=True)
tweet_csv.show(truncate=False, n=3)

+------+---------+---------------+---------------------------------+
|ItemID|Sentiment|SentimentSource|SentimentText                    |
+------+---------+---------------+---------------------------------+
|1038  |1        |Sentiment140   |that film is fantastic #brilliant|
|1804  |1        |Sentiment140   |this music is really bad #myband |
|1693  |0        |Sentiment140   |winter is terrible #thumbs-down  |
+------+---------+---------------+---------------------------------+
only showing top 3 rows



In [6]:
data=tweet_csv.select(
    "SentimentText",col("Sentiment").cast("Int").alias("label"))
data.show(truncate=False, n=5)

+---------------------------------+-----+
|SentimentText                    |label|
+---------------------------------+-----+
|that film is fantastic #brilliant|1    |
|this music is really bad #myband |1    |
|winter is terrible #thumbs-down  |0    |
|this game is awful #nightmare    |0    |
|I love jam #loveit               |1    |
+---------------------------------+-----+
only showing top 5 rows



In [7]:
#Spliting training and Testing
split_data=data.randomSplit([0.7,0.3])
train=split_data[0]

#label in test renamed to true label
test=split_data[1].withColumnRenamed("label","true_label")
train_rows=train.count()
test_rows=test.count()

print("Total train :",train_rows)
print("Total test :", test_rows)

Total train : 1349
Total test : 583


In [8]:
#Prepare data for training
tokenizer = Tokenizer(inputCol="SentimentText", outputCol="SentimentWords")
tokenizedTrain = tokenizer.transform(train)
tokenizedTrain.show(truncate=False, n=5)

+-------------------------+-----+------------------------------+
|SentimentText            |label|SentimentWords                |
+-------------------------+-----+------------------------------+
|I adore cheese #bestever |1    |[i, adore, cheese, #bestever] |
|I adore cheese #brilliant|1    |[i, adore, cheese, #brilliant]|
|I adore cheese #favorite |1    |[i, adore, cheese, #favorite] |
|I adore cheese #loveit   |1    |[i, adore, cheese, #loveit]   |
|I adore cheese #thumbs-up|1    |[i, adore, cheese, #thumbs-up]|
+-------------------------+-----+------------------------------+
only showing top 5 rows



In [9]:
swr = StopWordsRemover(inputCol=tokenizer.getOutputCol(), 
                       outputCol="MeaningfulWords")
SwRemovedTrain = swr.transform(tokenizedTrain)
SwRemovedTrain.show(truncate=False, n=5)

+-------------------------+-----+------------------------------+---------------------------+
|SentimentText            |label|SentimentWords                |MeaningfulWords            |
+-------------------------+-----+------------------------------+---------------------------+
|I adore cheese #bestever |1    |[i, adore, cheese, #bestever] |[adore, cheese, #bestever] |
|I adore cheese #brilliant|1    |[i, adore, cheese, #brilliant]|[adore, cheese, #brilliant]|
|I adore cheese #favorite |1    |[i, adore, cheese, #favorite] |[adore, cheese, #favorite] |
|I adore cheese #loveit   |1    |[i, adore, cheese, #loveit]   |[adore, cheese, #loveit]   |
|I adore cheese #thumbs-up|1    |[i, adore, cheese, #thumbs-up]|[adore, cheese, #thumbs-up]|
+-------------------------+-----+------------------------------+---------------------------+
only showing top 5 rows



In [10]:
hashTF = HashingTF(inputCol=swr.getOutputCol(), outputCol="features")
numericTrain = hashTF.transform(SwRemovedTrain).select(
    'label', 'MeaningfulWords', 'features')
numericTrain.show(truncate=False, n=3)

+-----+---------------------------+-------------------------------------------+
|label|MeaningfulWords            |features                                   |
+-----+---------------------------+-------------------------------------------+
|1    |[adore, cheese, #bestever] |(262144,[1689,91011,100089],[1.0,1.0,1.0]) |
|1    |[adore, cheese, #brilliant]|(262144,[1689,45361,100089],[1.0,1.0,1.0]) |
|1    |[adore, cheese, #favorite] |(262144,[1689,100089,108624],[1.0,1.0,1.0])|
+-----+---------------------------+-------------------------------------------+
only showing top 3 rows



In [11]:
#Training Model
lr = LogisticRegression(labelCol="label", featuresCol="features", 
                        maxIter=10, regParam=0.01)
model = lr.fit(numericTrain)
print ("Training Done")

Training Done


In [12]:
#Prepare Testing data
tokenizedTest = tokenizer.transform(test)
SwRemovedTest = swr.transform(tokenizedTest)
numericTest = hashTF.transform(SwRemovedTest)
numericTest.show(truncate=False, n=2)

+---------------------------------+----------+---------------------------------------+------------------------------------+-------------------------------------------------------+
|SentimentText                    |true_label|SentimentWords                         |MeaningfulWords                     |features                                               |
+---------------------------------+----------+---------------------------------------+------------------------------------+-------------------------------------------------------+
|I adore cheese #toptastic        |1         |[i, adore, cheese, #toptastic]         |[adore, cheese, #toptastic]         |(262144,[1689,42010,100089],[1.0,1.0,1.0])             |
|I adore classical music #bestever|1         |[i, adore, classical, music, #bestever]|[adore, classical, music, #bestever]|(262144,[91011,100089,102383,131250],[1.0,1.0,1.0,1.0])|
+---------------------------------+----------+---------------------------------------+--------------

In [13]:
#Prediction
raw_prediction = model.transform(numericTest)
raw_prediction.printSchema()


root
 |-- SentimentText: string (nullable = true)
 |-- true_label: integer (nullable = true)
 |-- SentimentWords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- MeaningfulWords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [14]:
Final_prediction = raw_prediction.select("MeaningfulWords", "prediction", "true_label")
Final_prediction.show(n=4, truncate = False)


+-------------------------------------+----------+----------+
|MeaningfulWords                      |prediction|true_label|
+-------------------------------------+----------+----------+
|[adore, cheese, #toptastic]          |1.0       |1         |
|[adore, classical, music, #bestever] |1.0       |1         |
|[adore, classical, music, #brilliant]|1.0       |1         |
|[adore, classical, music, #toptastic]|1.0       |1         |
+-------------------------------------+----------+----------+
only showing top 4 rows



In [15]:
Total_True=Final_prediction.filter(Final_prediction['prediction']==Final_prediction['true_label']).count()
Alldata=Final_prediction.count()
Accuracy=Total_True/Alldata
print("Accuracy Score is:", Accuracy*100, '%')

Accuracy Score is: 99.14236706689536 %
